In [14]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import random
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image


In [16]:
bs = 128

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [17]:
def train(epoch, device, weight):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var, weight)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [18]:
def test(device, weight):
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var, weight).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

# Task 1: Design the autoencoder structured network for MNIST

In [19]:
# YOUR CODE!!

class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        
        #############################################################
    
        # YOUR CODE!!
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc3_1 = nn.Linear(h_dim2, z_dim)
        self.fc3_2 = nn.Linear(h_dim2, z_dim)

        #############################################################
        
        
        # decoder part
        
        #############################################################
    
        # YOUR CODE!!
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        #############################################################
        
        
    def encoder(self, x):
        # return mu, log_var
        
        #############################################################
    
        # YOUR CODE!!
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc3_1(h), self.fc3_2(h) # mu, log_var
        #############################################################
    
    def sampling(self, mu, log_var):
        # return z sample
        
        #############################################################
    
        # YOUR CODE!!
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        #############################################################
        
    def decoder(self, z):
        # return generated img
        
        #############################################################
    
        # YOUR CODE!!
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h))
        #############################################################
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=50)
if torch.cuda.is_available():
    vae.to(device)

In [22]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3_1): Linear(in_features=256, out_features=50, bias=True)
  (fc3_2): Linear(in_features=256, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

# Task 2: Design the loss function for autoencoder with weight of KLD term

In [23]:
optimizer = optim.Adam(vae.parameters())

In [24]:

def loss_function(recon_x, x, mu, log_var, weight):
    # return reconstruction error + KL divergence losses
    
    #############################################################
    
    # YOUR CODE!!
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + weight*KLD
    #############################################################
    
    pass

In [25]:
from tqdm.autonotebook import tqdm

weight = 1.0

for epoch in tqdm(range(1, 51)):
    train(epoch, device, weight)
    test(device, weight)

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.653625
Train Epoch: 1 [12800/60000 (21%)]	Loss: 199.833420
Train Epoch: 1 [25600/60000 (43%)]	Loss: 175.766281
Train Epoch: 1 [38400/60000 (64%)]	Loss: 160.482452
Train Epoch: 1 [51200/60000 (85%)]	Loss: 149.153885
====> Epoch: 1 Average loss: 178.2026
====> Test set loss: 142.8234
Train Epoch: 2 [0/60000 (0%)]	Loss: 143.130768
Train Epoch: 2 [12800/60000 (21%)]	Loss: 134.891724
Train Epoch: 2 [25600/60000 (43%)]	Loss: 139.952286
Train Epoch: 2 [38400/60000 (64%)]	Loss: 129.791550
Train Epoch: 2 [51200/60000 (85%)]	Loss: 123.460632
====> Epoch: 2 Average loss: 132.3210
====> Test set loss: 123.7309
Train Epoch: 3 [0/60000 (0%)]	Loss: 126.979462
Train Epoch: 3 [12800/60000 (21%)]	Loss: 122.342552
Train Epoch: 3 [25600/60000 (43%)]	Loss: 122.273315
Train Epoch: 3 [38400/60000 (64%)]	Loss: 115.168045
Train Epoch: 3 [51200/60000 (85%)]	Loss: 113.036797
====> Epoch: 3 Average loss: 119.9631
====> Test set loss: 115.4444
Train Epoch: 4 [0/60000 (0%)]	L

Train Epoch: 26 [25600/60000 (43%)]	Loss: 97.691620
Train Epoch: 26 [38400/60000 (64%)]	Loss: 96.240356
Train Epoch: 26 [51200/60000 (85%)]	Loss: 99.888481
====> Epoch: 26 Average loss: 99.1719
====> Test set loss: 100.3470
Train Epoch: 27 [0/60000 (0%)]	Loss: 98.523796
Train Epoch: 27 [12800/60000 (21%)]	Loss: 93.197357
Train Epoch: 27 [25600/60000 (43%)]	Loss: 98.858337
Train Epoch: 27 [38400/60000 (64%)]	Loss: 98.419098
Train Epoch: 27 [51200/60000 (85%)]	Loss: 97.275337
====> Epoch: 27 Average loss: 99.0315
====> Test set loss: 100.2639
Train Epoch: 28 [0/60000 (0%)]	Loss: 96.671700
Train Epoch: 28 [12800/60000 (21%)]	Loss: 95.544449
Train Epoch: 28 [25600/60000 (43%)]	Loss: 97.994720
Train Epoch: 28 [38400/60000 (64%)]	Loss: 96.478973
Train Epoch: 28 [51200/60000 (85%)]	Loss: 95.185966
====> Epoch: 28 Average loss: 98.8812
====> Test set loss: 100.3864
Train Epoch: 29 [0/60000 (0%)]	Loss: 102.414253
Train Epoch: 29 [12800/60000 (21%)]	Loss: 94.755432
Train Epoch: 29 [25600/60000 (

# Task 3

In [26]:
with torch.no_grad():
    
    #########################################################
    
    # YOUR CODE!!
    z = torch.randn(64, 50).to(device)
    sample = vae.decoder(z).to(device)
    
    
    #########################################################
    
            
    #z = torch.tensor(z).to(device)
    #sample = vae.decoder(z.float())
    
    if not os.path.exists('./samples'):
        os.makedirs('./samples')
    
    save_image(sample.view(64, 1, 28, 28), './samples/problem 2/sample_50D' + '.png')